In [1]:
import pandas as pd
import numpy as np
import re
import datetime
from time import gmtime, strftime

df = pd.read_excel("Copy of NALSA 2010 Responses.xlsx", skiprows=1).fillna("NR")

In [2]:
# renaming the column names in the NALSA 2010 file 
df_colnames = pd.read_excel("DataDict_NALSA2010.xlsx")
colnames =  list(df_colnames['COL_NAME'].unique())[:6] + list(df_colnames['COL_NAME'].unique())[7:]

df.columns = colnames
df = df[[col for col in df.columns if not col.startswith("REM")]]

In [3]:
# shape of the dataset
df.shape

(244, 30)

In [4]:
# top 5 rows 
df.head()

,STATE,DISTRICTS,NUM_LWYR_PANEL,NUM_LWYR_RETAINER,NUM_LWYR_PANEL_All,SEP_PANELS,NUM_APLCTN_PRISON,NUM_APLCTN_COURT,NUM_LWYR_APNTD_PRISON,NUM_LWYR_APNTD_COURT,...,NUM_WTDRW,MNTRNG_COMTT,NUM_MNTRNG_COMTT_STAFF,NUM_RPRT_PNL_TO_MNTRNG,NUM_RPRT_MNTRNG,NUM_CASE_CMPLN_PNL,NUM_LWYR_REM_PNL,SCRUTINY_COMTT,DATE_MONTRNG_COMTT,DESIG_MEM
0,West Bengal,BALURGHAT,102,0,"S-70, L-0,JO-0.M-7,LP-0",Yes\n,322,212,322,212,...,0,Yes,0,None,0,No complaints received,Nil,No,28-03-2016,JSA
1,HIMACHAL PRADESH,BILASPUR,Not Provided,Not provided,7,Yes,1190,1190,IN ALL THE CASES,All,...,Not provided,Yes,0,None,Not provided,No complaints received,Nil,YES,2016-08-08 00:00:00,JSA
2,HIMACHAL PRADESH,MANDI,Not responded,Not responded,Not Responded,Not Provided,Not Responded,Not Responded,Not Responded,Not Responded,...,Not responded,Not Responded,Not responded,Not Responded,Not Responded,Not Responded,Not Responded,Not provided,Not Responded,Not responded
3,HIMACHAL PRADESH,To clarify the name,96,15,S-36,No,9,26,6,26,...,4,Yes,Not responded,21,12,No complaints received,Nil,YES,2014-05-11 00:00:00,Not provided
4,HIMACHAL PRADESH,SIRMAUR,22,Not responded,Not Responded,No,0,17,0,13,...,Not responded,Yes,Not responded,Not Responded,Not Responded,No complaints received,Nil,YES,2014-04-11 00:00:00,JSA


In [5]:
# list of columns
df.columns

Index(['STATE', 'DISTRICTS', 'NUM_LWYR_PANEL', 'NUM_LWYR_RETAINER',
       'NUM_LWYR_PANEL_All', 'SEP_PANELS', 'NUM_APLCTN_PRISON',
       'NUM_APLCTN_COURT', 'NUM_LWYR_APNTD_PRISON', 'NUM_LWYR_APNTD_COURT',
       'NUM_LWYR_APNTD_PRISON_NOT1', 'NUM_LWYR_APNTD_COURT_NOT1',
       'NUM_LWYR_APNTD_PRISON_NOT2', 'NUM_LWYR_APNTD_COURT_NOT2',
       'NUM_ACSD_RPSNTD_PANEL', 'NUM_ACSD_RPSNTD_RETAINER', 'NUM_BAIL_REQ',
       'NUM_ACQTL', 'NUM_DSPSD', 'NUM_CMPLT_REPORT', 'NUM_WTDRW',
       'MNTRNG_COMTT', 'NUM_MNTRNG_COMTT_STAFF', 'NUM_RPRT_PNL_TO_MNTRNG',
       'NUM_RPRT_MNTRNG', 'NUM_CASE_CMPLN_PNL', 'NUM_LWYR_REM_PNL',
       'SCRUTINY_COMTT', 'DATE_MONTRNG_COMTT', 'DESIG_MEM'],
      dtype='object')

In [6]:
rule_dict = {"NA":['N.A','NOT APPLICABLE','NOT APPLICABLE ','NA ','Not Applicable','Not Applicable ','N.A ','Not applicable'],
             "NP":[' nan,','nan',' nan','Nil','ATTACHED','Cumm','Cumm Response','Data not collated','DATA NOT MAINTAINED',
                                      'DOES NOT ARISE','INFO. PERTAINS TO COURTS','INFOMATION PERTAINS TO COURTS',
                                      'INFORMATION NOT AVAILABLE','NIL','No','NO MAINTAINED','NO OF STAFF PROVIDED',
                                      'NO RECORD AVAILBALE','NO SUCH DATA MAINTAINED','None','Not available','NOT COMPILED',
                                      'Not constituted','NOT COUNTED','NOT KNOWN','NOT MAINTAINED','Not Provided',
                                      'NOT REPORTED','NOT SUBMITTED BY PANEL LAWYER','PENDING','RECVIED THROUGH SDLSCs',
                                      'REPORT NOT RECEIVED','Response not clear','ATTACHED ',
                                      'THE SERVICE OF JUNIOR ADMINISTRATIVE ASSISTANT WHO ATTACHED TO DLSAs ABOVE IS UTILIZED',
                                      'To check','To check attachment','Not provided','NOT AVAILABLE','NOT COMPILED ',
                   'NOT AVAILABLE ','NOT COUNTED ','NOT KNOWN ','INFOMATION PERTAINS TO COURTS ','PENDING ','NOT REPORTED ',
                   'REPORT NOT RECEIVED ','INFORMATION NOT AVAILABLE ','DATA NOT COLLATED ', 'NOT SUBMITTED BY PANEL LAWYER '], 
             "NR":['No Response','Not responded','Not Responded']}

In [7]:
# creationg a dictionary of column names and there string values
str_values_by_columns = {}

for cols in df.columns:
    str_values_by_columns[cols] = [val for val in df[cols].unique() if not str(val).isdigit()]
    
# list of sub-strings to be mapped as NA values - 0/Yes - 1/monthly - 12/ bimonthly - 6/ Quarterly - 4 
##nvals = ['NOT','N.A','NO','NIL','NAN','ATTACH','CHECK','PENDING','PERTAIN','CUMM']
yvals = ['YES']
monthvals = ['MONTH', 'REGULAR', 'RECIEVE', 'MAINTAIN', 'RECVIED', 'CONSTIUTED']
quartvals = ['QUART','QUATER']
bimonvals = ['BI']
# list of strings containing the sub-strings (to be replaced in data)
#na_str_val = []
yes_str_val = []
mon_str_val = []
quart_str_val = []
bimon_str_val = []

# creating list of strings to be replaced
for k in str_values_by_columns.keys():
    if k not in ['STATE','DISTRICTS','NUM_LWYR_PANEL_SR','NUM_MNTRNG_COMTT_STAFF']:
        #na_str_val = na_str_val + [x for x in str_values_by_columns[k] if any(nv.lower() in str(x).lower() for nv in nvals)]
        yes_str_val = yes_str_val + [x for x in str_values_by_columns[k] if any(yv.lower() in str(x).lower() for yv in yvals)]
        mon_str_val = mon_str_val + [x for x in str_values_by_columns[k] if any(mv.lower() in str(x).lower() for mv in monthvals)]
        bimon_str_val = bimon_str_val + [x for x in str_values_by_columns[k] if any(bv.lower() in str(x).lower() for bv in bimonvals)]
        quart_str_val = quart_str_val + [x for x in str_values_by_columns[k] if any(qv.lower() in str(x).lower() for qv in quartvals)]

# replacing the the list of string with there respective values
for cols in df.columns:
    #replacing strings with NA values to 0
    ##df[cols][df[cols].isin(list(set(na_str_val)))] = 0
    for k in rule_dict.keys():
        df[cols][df[cols].isin(rule_dict[k])] = k
    #replacing strings with YES values to 1
    df[cols][df[cols].isin(list(set(yes_str_val)))] = 1
    #replacing strings with monthly values to 12
    df[cols][df[cols].isin(list(set(mon_str_val)))] = 12
    #replacing strings with  bi-monthly values to 6
    df[cols][df[cols].isin(list(set(bimon_str_val)))] = 6
    #replacing strings with quarterly values to4
    df[cols][df[cols].isin(list(set(quart_str_val)))] = 4
    
# treating few outlier cases seperately
df.NUM_LWYR_APNTD_COURT[(df.NUM_LWYR_APNTD_COURT.isin(['All']))] = df.NUM_APLCTN_COURT
df.NUM_LWYR_APNTD_PRISON[(df.NUM_LWYR_APNTD_PRISON.isin(['IN ALL THE CASES']))] = df.NUM_APLCTN_PRISON
df.NUM_LWYR_APNTD_COURT_NOT1[(df.NUM_LWYR_APNTD_COURT_NOT1.isin(['IN 9 CASES LEGAL AID PROVIDED']))] = 0
df.NUM_LWYR_APNTD_PRISON_NOT1[(df.NUM_LWYR_APNTD_PRISON_NOT1.isin(['IN 9 CASES LEGAL AID PROVIDED']))] = 0
df.NUM_ACQTL[(df.NUM_ACQTL.isin(['R11 (1) OF NALSA 2010-(267)']))] = 0

In [8]:
# function calculates the sum of all numbers in a strin (i.e. composition of Panel/Monitoring comitte)
def calcNum(panelVal):
    if type(panelVal) == int:
        return panelVal
    else:
        return sum([int(s) for s in re.findall(r'\b\d+\b', panelVal)])

In [9]:
# NUM_LWYR_PANEL_TOT & NUM_MNTRNG_COMTT_STAFF contain sum of total number of members on the panels/committe
df['NUM_LWYR_PANEL_TOT'] = df.NUM_LWYR_PANEL_All.apply(lambda x: calcNum(x))
df.NUM_MNTRNG_COMTT_STAFF = df.NUM_MNTRNG_COMTT_STAFF.apply(lambda x: calcNum(x))

In [10]:
# replacing all strings other than NP NR NA withe zero
for cols in df.columns:
    if cols not in ['STATE','DISTRICTS','NUM_LWYR_PANEL_All','DATE_MONTRNG_COMTT',]:
        df[cols][df[cols].isin([val for val in df[cols].unique() if not str(val).isdigit() and val not in ['NP','NR','NA']])] = 0    

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [11]:
# extracting year from monitoring committee constitution date
def yearMNTRNGCOMTT(x):
    if type(x) == datetime.datetime:
        return x.year
    elif (type(x)!= int) and (re.search(r'-|/' ,x)):
        return "20"+str(re.sub(r' |,','',x)[-2:])
    else:
        return x
        
df['YR_MNTRNG_COMTT'] = df.DATE_MONTRNG_COMTT.apply(lambda x : yearMNTRNGCOMTT(x))

In [12]:
# checking for any un-treated string values in the dataset
str_values_by_columns_check = {}
for cols in df.columns:
    str_values_by_columns_check[cols] = [val for val in df[cols].unique() if not str(val).isdigit() ]
for k in str_values_by_columns_check.keys():
    if k not in ['STATE','DISTRICTS','NUM_LWYR_PANEL_All','DATE_MONTRNG_COMTT']:
        print (k, str_values_by_columns_check[k])

NUM_LWYR_PANEL ['NP', 'NR']
NUM_LWYR_RETAINER ['NP', 'NR']
SEP_PANELS ['NP', 'NR']
NUM_APLCTN_PRISON ['NR', 'NP']
NUM_APLCTN_COURT ['NR', 'NP']
NUM_LWYR_APNTD_PRISON ['NR', 'NP']
NUM_LWYR_APNTD_COURT ['NR', 'NP']
NUM_LWYR_APNTD_PRISON_NOT1 ['NR']
NUM_LWYR_APNTD_COURT_NOT1 ['NR']
NUM_LWYR_APNTD_PRISON_NOT2 ['NR', 'NA']
NUM_LWYR_APNTD_COURT_NOT2 ['NR', 'NP', 'NA']
NUM_ACSD_RPSNTD_PANEL ['NR', 'NP']
NUM_ACSD_RPSNTD_RETAINER ['NP', 'NR']
NUM_BAIL_REQ ['NP', 'NR']
NUM_ACQTL ['NP', 'NR', 'NA']
NUM_DSPSD ['NP', 'NR']
NUM_CMPLT_REPORT ['NP', 'NR', 'NA']
NUM_WTDRW ['NP', 'NR']
MNTRNG_COMTT ['NR', 'NP']
NUM_MNTRNG_COMTT_STAFF []
NUM_RPRT_PNL_TO_MNTRNG ['NP', 'NR', 'NA']
NUM_RPRT_MNTRNG ['NP', 'NR']
NUM_CASE_CMPLN_PNL ['NR', 'NP', 'NA']
NUM_LWYR_REM_PNL ['NP', 'NR', 'NA']
SCRUTINY_COMTT ['NP', 'NR']
DESIG_MEM ['NR', 'NP', 'NA']
NUM_LWYR_PANEL_TOT []
YR_MNTRNG_COMTT ['NR', 'NP', 'NA']


In [13]:
df.drop(['NUM_LWYR_PANEL_All','DATE_MONTRNG_COMTT'],inplace=True, axis=1)
df.drop_duplicates(['DISTRICTS'], inplace=True)
# exporting to csv
df.to_csv("NALSA_2010_cleaned_"+re.sub(r' |:', '-', strftime("%Y-%m-%d %H:%M:%S", gmtime()))+".csv")
df.head()

,STATE,DISTRICTS,NUM_LWYR_PANEL,NUM_LWYR_RETAINER,SEP_PANELS,NUM_APLCTN_PRISON,NUM_APLCTN_COURT,NUM_LWYR_APNTD_PRISON,NUM_LWYR_APNTD_COURT,NUM_LWYR_APNTD_PRISON_NOT1,...,MNTRNG_COMTT,NUM_MNTRNG_COMTT_STAFF,NUM_RPRT_PNL_TO_MNTRNG,NUM_RPRT_MNTRNG,NUM_CASE_CMPLN_PNL,NUM_LWYR_REM_PNL,SCRUTINY_COMTT,DESIG_MEM,NUM_LWYR_PANEL_TOT,YR_MNTRNG_COMTT
0,West Bengal,BALURGHAT,102,0,1,322,212,322,212,0,...,1,0,NP,0,0,NP,NP,0,77,2016
1,HIMACHAL PRADESH,BILASPUR,NP,NP,1,1190,1190,1190,1190,12,...,1,0,NP,NP,0,NP,1,0,7,2016
2,HIMACHAL PRADESH,MANDI,NR,NR,NP,NR,NR,NR,NR,NR,...,NR,0,NR,NR,NR,NR,NP,NR,0,NR
3,HIMACHAL PRADESH,To clarify the name,96,15,NP,9,26,6,26,0,...,1,0,21,12,0,NP,1,NP,36,2014
4,HIMACHAL PRADESH,SIRMAUR,22,NR,NP,0,17,0,13,0,...,1,0,NR,NR,0,NP,1,0,0,2014
